# 掩码语言模型训练实例

MLM任务的预训练！

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

## Step1 导入相关包

In [2]:
from datasets import load_dataset, Dataset
# AutoModelForMaskedLM好像之前都没有使用过？掩码语言模型可能就是要用DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")

In [4]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [5]:
ds[0] # MLM任务所以不需要标签，它自己就是标签

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## Step3 数据集处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("/data/PLM/chinese-macbert-base")

def process_func(examples):
    # 这里只要编码就行了，掩码和padding用DataLoader在DataCollatorForLanguageModeling中会完成的
    return tokenizer(examples["completion"], max_length=384, truncation=True) 

In [7]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [8]:
from torch.utils.data import DataLoader

# padding到每个batch中的最大长度
dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15))

In [9]:
next(enumerate(dl)) # 遍历一个batch

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(0,
 {'input_ids': tensor([[  101,  6205,  2128,   769,  6858,  1920,  2110,  1300,  4289,  7667,
           8020, 13135,   112,  9064, 12095,  8731,   103,  8181,  8736, 10553,
           8021,  3221,   671,  2429,   855,   754,  6205,  2128,   769,  6858,
           1920,  2110,   103,  1300,  3667,  7667,  8024,  7667,  7270,  3221,
           7247,  3209,  1587,   511,  1325,  1380,  8258,  2399,   130,  3299,
           8113,  3189,  2458,  1993,  5040,  2456,  8024,  8138,   103,   125,
           3299,   129,  3189,  3633,  2466,  2456,   103,  2458,  7667,  8024,
            855,   754,  6205,  2128,   103,  6858,  1920,  2110,  1069,  2412,
           3413,  1277,  7362,  6205,  4689,  6205,  2128,  2356,   103,  2123,
           6205,  6662,   103,  1384,   511,  2456,  5029,   103,  4916,   127,
            117,  8280,  2398,  5101,  8024,  2245,  1324,  7481,  4916,   125,
            117,  8195,  2398,  5101,  8024,  7667,  5966,  3152,  4289,   125,
            117,  8567

In [10]:
tokenizer.mask_token, tokenizer.mask_token_id # 只有被[MASK]掉的部分会有标签，其他labels都用-100填充

('[MASK]', 103)

## Step4 创建模型

In [11]:
model = AutoModelForMaskedLM.from_pretrained("/data/PLM/chinese-macbert-base")
model # 最后不再是BertPooler+Dropout+Classifier！改成了BertOnlyMLMHead

Some weights of the model checkpoint at /data/PLM/chinese-macbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

## Step5 配置训练参数

In [12]:
args = TrainingArguments(
    output_dir="./masked_lm",
    per_device_train_batch_size=32,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [13]:
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds,
    # 上面那个DataCollatorForLanguageModeling只是取样查看，真正的DataCollatorForLanguageModeling要用在这！
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15)
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step7 模型训练

In [14]:
trainer.train()

Step,Training Loss
10,1.422400
20,1.338600
30,1.387100
40,1.356000
50,1.390600
60,1.335300
70,1.328200
80,1.329600
90,1.372900
100,1.372800


TrainOutput(global_step=313, training_loss=1.3310567708061145, metrics={'train_runtime': 260.7634, 'train_samples_per_second': 38.349, 'train_steps_per_second': 1.2, 'total_flos': 1973819658240000.0, 'train_loss': 1.3310567708061145, 'epoch': 1.0})

## Step8 模型推理

In [15]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0) # fill-mask任务只需要带[MASK]的一串字符就行！

In [16]:
# 有两个[MASK]的话会出现嵌套列表：内层列表有两个，各预测一个[MASK]，而保持另一个[MASK]；可以各选两者中最大的，或者进行组合得到最终结果
pipe("西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

[[{'score': 0.9977889060974121,
   'token': 1920,
   'token_str': '大',
   'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0012657546903938055,
   'token': 2110,
   'token_str': '学',
   'sequence': "[CLS] 西 安 交 通 学 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 9.880658035399392e-05,
   'token': 4906,
   'token_str': '科',
   'sequence': "[CLS] 西 安 交 通 科 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 5.597392009804025e-05,
   'token': 1325,
   'token_str': '历',
   'sequence': "[CLS] 西 安 交 通 历 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 5.036651054979302e-05,
   'token': 2339,
   'token_str': '工',
   'sequence': "[CLS] 西 安 交 通 工 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}],
 [{'score': 0.9990832805633

In [17]:
pipe("下面是一则[MASK][MASK]新闻。小编报道，近日，游戏产业发展的非常好！") # 默认输出5个结果

[[{'score': 0.0840931236743927,
   'token': 7028,
   'token_str': '重',
   'sequence': '[CLS] 下 面 是 一 则 重 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.057448919862508774,
   'token': 2031,
   'token_str': '娱',
   'sequence': '[CLS] 下 面 是 一 则 娱 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.03312835469841957,
   'token': 3300,
   'token_str': '有',
   'sequence': '[CLS] 下 面 是 一 则 有 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.031283412128686905,
   'token': 4178,
   'token_str': '热',
   'sequence': '[CLS] 下 面 是 一 则 热 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.028763465583324432,
   'token': 4696,
   'token_str': '真',
   'sequence': '[CLS] 下 面 是 一 则 真 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'}],
 [{'score': 0.08967147022485733,
   'token': 7481,
   'token_str': '面',
   'sequence': '[CLS] 下 面 是 一 则 [MASK] 面 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
 